In [4]:
import pickle, os, gzip, json, sys, itertools
from pathlib import Path
from importlib import reload
from dataclasses import dataclass, field
import collections
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pysam
import scipy as sp
import seaborn
import sharedmem

plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["figure.dpi"] = 300


sys.path.append("scripts")
sys.path.append("../../scripts")

In [5]:
from data_io import is_fwd_id, get_fwd_id, get_sibling_id
from dim_reduction import SpectralEmbedding, scBiMapEmbedding
from nearest_neighbors import (
    ExactNearestNeighbors,
    NNDescent,
    WeightedLowHash,
    PAFNearestNeighbors,
    LowHash,
    HNSW,
    ProductQuantization,
    _NearestNeighbors,
    IVFProductQuantization,
)
from graph import OverlapGraph, GenomicInterval, get_overlap_statistics, remove_false_edges
from truth import get_overlaps
from evaluate import NearestNeighborsConfig, mp_compute_nearest_neighbors

/home/miaochangjiu/kNN-overlap-finder/scripts/../lib


## Parameters


In [6]:
MAX_SAMPLE_SIZE = int(1e9)
COVERAGE_DEPTH = 20

## Load data

In [7]:

sample = snakemake.wildcards['sample']
dataset = snakemake.wildcards['platform']
region = snakemake.wildcards['region']
method = snakemake.wildcards['method']

if method =="Minimap2":
    paf_path = snakemake.input['paf_minimap2']
elif method == "Blend":
    paf_path = snakemake.input['paf_blend']
else:
    paf_path = ''

npz_path = snakemake.input['feature_matrix']
tsv_path = snakemake.input['metadata']
json_path = snakemake.input['read_features']

nbr_path = snakemake.output['nbr_indice']
stat_path = snakemake.output['overlap']
threads  = snakemake.threads

print(sample, dataset, region)

NameError: name 'snakemake' is not defined

In [ ]:
meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

feature_weights = {i: 1 for i in range(feature_matrix.shape[1])}

In [ ]:
fig, ax = plt.subplots(figsize=(8, 2.5))
ax.hist([len(x) for x in read_features.values()], bins=100)
ax.set_xlim(left=0)
ax.set_xlabel("Number of features per read")
ax.set_ylabel("Number of reads")
ax.grid(color='k', alpha=0.1)

## Get nearest neighbours

In [ ]:
kw = dict(data=feature_matrix)
max_bucket_size = COVERAGE_DEPTH * 1.5
max_n_neighbors = COVERAGE_DEPTH

with open('workflow/notebooks/config_dict.pkl', 'rb') as file:  
    config_dict = pickle.load(file)

config_dict["Minimap2"]=NearestNeighborsConfig(
        nearest_neighbors_method=PAFNearestNeighbors,
        description="Minimap2 all-vs-all",
        nearest_neighbors_kw=dict(paf_path=paf_path, read_indices=read_indices)
        )
config_dict["Blend"]=NearestNeighborsConfig(
        nearest_neighbors_method=PAFNearestNeighbors,
        description="Blend",
        nearest_neighbors_kw=dict(paf_path=paf_path, read_indices=read_indices)
        )  

In [ ]:
processes = threads
config = config_dict[method]
configs = [config]
kw = dict(data=feature_matrix)

nbr_dict, time_dict, memory_dict = mp_compute_nearest_neighbors(
    data=feature_matrix,
    configs=configs,
    n_neighbors=max_n_neighbors,
    processes=processes,
)

np.savez(nbr_path, nbr_dict)

## Statistics

In [ ]:
read_ids = np.array(list(read_features))
graphs = collections.defaultdict(dict)
k_values = np.arange(2, max_n_neighbors + 1)
for i, config in enumerate(configs):
    print(i, end=" ")
    nbr_indices = nbr_dict[i]
    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graphs[i][k] = graph

In [ ]:
def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)
len(read_intervals)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count


In [1]:
df_rows = []
for i, config in enumerate(configs):
    print(i, end=' ')
    for k in k_values:
        graph = graphs[i][k]
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        time_stats = {"elapsed_time:" + k: v for k, v in time_dict[i].items()}
        memory_stats = {"peak_memory:" + k: v for k, v in memory_dict[i].items()}
        
        stats = {"config_id": i, "description": config.description, "n_neighbors": k, 
                 **graph_stats, **time_stats, **memory_stats}
        df_rows.append(stats)
df = pd.DataFrame(df_rows)
df['connected_fraction'] = 1 - df['singleton_fraction']
df.to_csv(stat_path,  sep='\t')

NameError: name 'configs' is not defined